# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from texttable import Texttable
import latextable

In [3]:
import sys
sys.path.insert(1, '../sar_dirichlet')
import dirichlet_regression

In [4]:
from func_test import cos_similarity, rmse_aitchison

In [5]:
scipy.__version__

'1.10.1'

# Loading Dataset

In [6]:
reefdata = pd.read_csv('Data Dirichlet/2016_Reef_data_4cat.csv', sep=';')

In [7]:
# We remove points that are the only ones in their reef
counts = reefdata['reef_name'].value_counts()
reefdata = reefdata[reefdata['reef_name'].isin(counts[counts > 1].index)]
reefdata = reefdata.reset_index(drop=True)

In [8]:
X_reefdata = reefdata.iloc[:,4]
X_reefdata = np.array([[j] for j in X_reefdata])
Y_reefdata = np.array(reefdata.iloc[:,-4:])

In [9]:
Z_reefdata = np.ones(len(X_reefdata)).reshape((len(X_reefdata),1))

In [10]:
n_features = 1
n_classes = 4

In [11]:
X = np.array(X_reefdata)
Y = np.array(Y_reefdata)

X = StandardScaler().fit(X).transform(X)

In [12]:
Z = Z_reefdata
gamma_0 = [0.]

In [13]:
n,K = X.shape
J = Y.shape[1]

In [14]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0

    # Convert coordinates from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def compute_distance_matrix(df):
    n = len(df)
    dist_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            if i == j:
                dist = 0.0
            else:
                dist = haversine(df.at[i, 'lat'], df.at[i, 'lng'],
                                 df.at[j, 'lat'], df.at[j, 'lng'])
            dist_matrix[i][j] = dist
            dist_matrix[j][i] = dist  # symmetric

    return dist_matrix

def compute_cont_matrix(df):
    n = len(df)
    cont_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            dist = 0.
            if (i!=j) and (df.at[i, 'reef_name'] == df.at[j, 'reef_name']):
                dist = 1
            cont_matrix[i][j] = dist
            cont_matrix[j][i] = dist  # symmetric

    return cont_matrix

In [15]:
distance_matrix = compute_distance_matrix(reefdata)

In [16]:
W_cont = compute_cont_matrix(reefdata)

In [17]:
W_dist = np.copy(distance_matrix)
W_dist[W_dist>0] = 1/W_dist[W_dist>0]

In [18]:
# row-normalize
W_cont = W_cont/W_cont.sum(axis=1)[:,None]
W_dist = W_dist/W_dist.sum(axis=1)[:,None]

# Order 1

## Dirichlet model

In [30]:
%%time
list_r2_ns_1, list_rmse_ns_1, list_aic_ns_1, list_crossentropy_ns_1, list_similarity_ns_1 = [], [], [], [], []
list_r2_cont_1, list_rmse_cont_1, list_aic_cont_1, list_crossentropy_cont_1, list_similarity_cont_1 = [], [], [], [], []
list_r2_dist_1, list_rmse_dist_1, list_aic_dist_1, list_crossentropy_dist_1, list_similarity_dist_1 = [], [], [], [], []
list_rmse_a_ns_1, list_rmse_a_cont_1, list_rmse_a_dist_1 = [], [], []
list_rho_dist_1, list_rho_cont_1 = [], []

for i in range(n): 
    X_temp = np.delete(X,i,axis=0)
    Y_temp = np.delete(Y,i,axis=0)
    Z_temp = np.delete(Z,i,axis=0)
    
    Wss_cont = np.delete(W_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp)
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_ns_1.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_ns_1.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_ns_1.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_aic_ns_1.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*5)
        list_similarity_ns_1.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_ns_1.append(rmse_aitchison(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont)
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_cont_1.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_cont_1.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_cont_1.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_aic_cont_1.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
        list_similarity_cont_1.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_cont_1.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
        list_rho_cont_1.append(dirichRegressor.rho)
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist)
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_dist_1.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_dist_1.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_dist_1.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_aic_dist_1.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
        list_similarity_dist_1.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_dist_1.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
        list_rho_dist_1.append(dirichRegressor.rho)

Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_RED

<timed exec>:14: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


<timed exec>:14: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


...


CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH

<timed exec>:14: RuntimeWarning: invalid value encountered in divide



Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


<timed exec>:14: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimizati

In [28]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity", "AIC", "RMSE_A"])
text_r2 = str(np.round(np.mean(list_r2_ns_1),4))+" ("+str(np.round(np.std(list_r2_ns_1),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_1),4))+" ("+str(np.round(np.std(list_rmse_ns_1),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_1),4))+" ("+str(np.round(np.std(list_crossentropy_ns_1),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_1),4))+" ("+str(np.round(np.std(list_similarity_ns_1),4))+")"
text_aic = str(np.round(np.mean(list_aic_ns_1),4))+" ("+str(np.round(np.std(list_aic_ns_1),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_ns_1),4))+" ("+str(np.round(np.std(list_rmse_a_ns_1),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_cont_1),4))+" ("+str(np.round(np.std(list_r2_cont_1),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_1),4))+" ("+str(np.round(np.std(list_rmse_cont_1),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_1),4))+" ("+str(np.round(np.std(list_crossentropy_cont_1),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_1),4))+" ("+str(np.round(np.std(list_similarity_cont_1),4))+")"
text_aic = str(np.round(np.mean(list_aic_cont_1),4))+" ("+str(np.round(np.std(list_aic_cont_1),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_cont_1),4))+" ("+str(np.round(np.std(list_rmse_a_cont_1),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_dist_1),4))+" ("+str(np.round(np.std(list_r2_dist_1),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_1),4))+" ("+str(np.round(np.std(list_rmse_dist_1),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_1),4))+" ("+str(np.round(np.std(list_crossentropy_dist_1),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_1),4))+" ("+str(np.round(np.std(list_similarity_dist_1),4))+")"
text_aic = str(np.round(np.mean(list_aic_dist_1),4))+" ("+str(np.round(np.std(list_aic_dist_1),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_dist_1),4))+" ("+str(np.round(np.std(list_rmse_a_dist_1),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic, text_rmse_a])

print(table.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Model     | $R^2$     | RMSE     | Cross-   | Cos simi | AIC      | RMSE_A   |
|           |           |          | entropy  | larity   |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| Not       | 0.1334    | 0.103    | 1.1953   | 0.9392   | -298.524 | 1.7832   |
| spatial   | (0.0089)  | (0.0011) | (0.0047) | (0.0015) | 8        | (0.0247) |
|           |           |          |          |          | (3.8001) |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| Contiguit | 0.1332    | 0.103    | 1.1951   | 0.9393   | -296.969 | 1.7834   |
| y         | (0.0079)  | (0.001)  | (0.0051) | (0.0015) | 4        | (0.0213) |
|           |           |          |          |          | (3.9833) |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| Distance  | 0.1423    | 0.

In [36]:
# For the computation of the AIC, we take the full dataset

dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z)
aic_ns = -2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)+2*5
print('AIC not spatial:', aic_ns)

dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z, W=W_cont)
aic_cont = -2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)+2*6
print('AIC contiguity:', aic_cont, 'with an estimated rho of', dirichRegressor.rho)

dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z, W=W_dist)
aic_dist = -2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)+2*6
print('AIC distance:', aic_dist, 'with an estimated rho of', dirichRegressor.rho)


Optimization terminated successfully.
AIC not spatial: -305.71529125956397
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
AIC contiguity: -303.7152912554337 with an estimated rho of 0.35016080528031657
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
AIC distance: -304.26946244693085 with an estimated rho of 0.8030353134129231


In [48]:
dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z, W=W_cont)
dirichRegressor.compute_hessian(X, Y, Z=Z, W=W_cont)

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
-0.001097872244768768


In [50]:
print('Estimated variance:', np.sqrt(-np.linalg.inv(-dirichRegressor.hess)[-1,-1]))

0.033134155259622475


In [51]:
dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z, W=W_dist)
dirichRegressor.compute_hessian(X, Y, Z=Z, W=W_dist)

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH


In [53]:
print('Estimated variance:', np.sqrt(-np.linalg.inv(-dirichRegressor.hess)[-1,-1]))

Estimated variance: 0.002354044662771318


## Cross-entropy

In [21]:
%%time
list_r2_ns_1_ce, list_rmse_ns_1_ce, list_crossentropy_ns_1_ce, list_similarity_ns_1_ce = [], [], [], []
list_r2_cont_1_ce, list_rmse_cont_1_ce, list_crossentropy_cont_1_ce, list_similarity_cont_1_ce = [], [], [], []
list_r2_dist_1_ce, list_rmse_dist_1_ce, list_crossentropy_dist_1_ce, list_similarity_dist_1_ce = [], [], [], []
list_rmse_a_ns_1_ce, list_rmse_a_cont_1_ce, list_rmse_a_dist_1_ce = [], [], []
list_rho_dist_1_ce, list_rho_cont_1_ce = [], []

for i in range(n):  
    X_temp = np.delete(X,i,axis=0)
    Y_temp = np.delete(Y,i,axis=0)
    Z_temp = np.delete(Z,i,axis=0)
    
    Wss_cont = np.delete(W_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, loss='crossentropy')
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_ns_1_ce.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_ns_1_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_ns_1_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_similarity_ns_1_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_ns_1_ce.append(rmse_aitchison(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont, loss='crossentropy')
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_cont_1_ce.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_cont_1_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_cont_1_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_similarity_cont_1_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_cont_1_ce.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
        list_rho_cont_1_ce.append(dirichRegressor.rho)
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist, loss='crossentropy')
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_dist_1_ce.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_dist_1_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_dist_1_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_similarity_dist_1_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_dist_1_ce.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
        list_rho_dist_1_ce.append(dirichRegressor.rho)

Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization

<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide



Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.

<timed exec>:14: RuntimeWarning: invalid value encountered in divide



Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.

...


Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.

<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide



Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.

<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide



Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimizatio

In [22]:
print('Mean rho contiguity:', np.mean(list_rho_cont_1_ce))
print('Mean rho distance:', np.mean(list_rho_dist_1_ce))

Mean rho contiguity: 0.18928500002889673
Mean rho distance: 0.8500173346405993


In [52]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity"])
text_r2 = str(np.round(np.mean(list_r2_ns_1_ce),4))+" ("+str(np.round(np.std(list_r2_ns_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_1_ce),4))+" ("+str(np.round(np.std(list_rmse_ns_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_ns_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_1_ce),4))+" ("+str(np.round(np.std(list_similarity_ns_1_ce),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_cont_1_ce),4))+" ("+str(np.round(np.std(list_r2_cont_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_1_ce),4))+" ("+str(np.round(np.std(list_rmse_cont_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_cont_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_1_ce),4))+" ("+str(np.round(np.std(list_similarity_cont_1_ce),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_dist_1_ce),4))+" ("+str(np.round(np.std(list_r2_dist_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_1_ce),4))+" ("+str(np.round(np.std(list_rmse_dist_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_dist_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_1_ce),4))+" ("+str(np.round(np.std(list_similarity_dist_1_ce),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity])

print(table.draw())

+-------------+----------------+---------------+---------------+---------------+
| Model       | $R^2$          | RMSE          | Cross-entropy | Cos           |
|             |                |               |               | similarity    |
+-------------+----------------+---------------+---------------+---------------+
| Not spatial | 0.1923         | 0.1001        | 1.1824        | 0.9429        |
|             | (0.0082)       | (0.001)       | (0.0042)      | (0.0013)      |
+-------------+----------------+---------------+---------------+---------------+
| Contiguity  | 0.1912         | 0.1001        | 1.1823        | 0.9429        |
|             | (0.0067)       | (0.0009)      | (0.0045)      | (0.0012)      |
+-------------+----------------+---------------+---------------+---------------+
| Distance    | 0.2022         | 0.0994        | 1.1815        | 0.9434        |
|             | (0.0093)       | (0.001)       | (0.0042)      | (0.0013)      |
+-------------+-------------

In [53]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity", "RMSE_A"])
text_r2 = str(np.round(np.mean(list_r2_ns_1_ce),4))+" ("+str(np.round(np.std(list_r2_ns_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_1_ce),4))+" ("+str(np.round(np.std(list_rmse_ns_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_ns_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_1_ce),4))+" ("+str(np.round(np.std(list_similarity_ns_1_ce),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_ns_1_ce),4))+" ("+str(np.round(np.std(list_rmse_a_ns_1_ce),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_cont_1_ce),4))+" ("+str(np.round(np.std(list_r2_cont_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_1_ce),4))+" ("+str(np.round(np.std(list_rmse_cont_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_cont_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_1_ce),4))+" ("+str(np.round(np.std(list_similarity_cont_1_ce),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_cont_1_ce),4))+" ("+str(np.round(np.std(list_rmse_a_cont_1_ce),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_dist_1_ce),4))+" ("+str(np.round(np.std(list_r2_dist_1_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_1_ce),4))+" ("+str(np.round(np.std(list_rmse_dist_1_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_1_ce),4))+" ("+str(np.round(np.std(list_crossentropy_dist_1_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_1_ce),4))+" ("+str(np.round(np.std(list_similarity_dist_1_ce),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_dist_1_ce),4))+" ("+str(np.round(np.std(list_rmse_a_dist_1_ce),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity, text_rmse_a])

print(table.draw())

+-------------+------------+------------+------------+------------+------------+
| Model       | $R^2$      | RMSE       | Cross-     | Cos        | RMSE_A     |
|             |            |            | entropy    | similarity |            |
+-------------+------------+------------+------------+------------+------------+
| Not spatial | 0.1923     | 0.1001     | 1.1824     | 0.9429     | 1.9157     |
|             | (0.0082)   | (0.001)    | (0.0042)   | (0.0013)   | (0.0287)   |
+-------------+------------+------------+------------+------------+------------+
| Contiguity  | 0.1912     | 0.1001     | 1.1823     | 0.9429     | 1.9169     |
|             | (0.0067)   | (0.0009)   | (0.0045)   | (0.0012)   | (0.0255)   |
+-------------+------------+------------+------------+------------+------------+
| Distance    | 0.2022     | 0.0994     | 1.1815     | 0.9434     | 1.9136     |
|             | (0.0093)   | (0.001)    | (0.0042)   | (0.0013)   | (0.0288)   |
+-------------+------------+

# Order 2

In [23]:
X_2 = np.ones((n,3))
X_2[:,1] = X[:,0]
X_2[:,2] = X[:,0]**2

## Dirichlet model

In [55]:
%%time
list_r2_ns_2, list_rmse_ns_2, list_aic_ns_2, list_crossentropy_ns_2, list_similarity_ns_2 = [], [], [], [], []
list_r2_cont_2, list_rmse_cont_2, list_aic_cont_2, list_crossentropy_cont_2, list_similarity_cont_2 = [], [], [], [], []
list_r2_dist_2, list_rmse_dist_2, list_aic_dist_2, list_crossentropy_dist_2, list_similarity_dist_2 = [], [], [], [], []
list_rmse_a_ns_2, list_rmse_a_cont_2, list_rmse_a_dist_2 = [], [], []

for i in range(n):  
    X_temp = np.delete(X_2,i,axis=0)
    Y_temp = np.delete(Y,i,axis=0)
    Z_temp = np.delete(Z,i,axis=0)
    
    Wss_cont = np.delete(W_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp)
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_ns_2.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_ns_2.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_ns_2.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_aic_ns_2.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*5)
        list_similarity_ns_2.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_ns_2.append(rmse_aitchison(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont)
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_cont_2.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_cont_2.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_cont_2.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_aic_cont_2.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
        list_similarity_cont_2.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_cont_2.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist)
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_dist_2.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_dist_2.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_dist_2.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_aic_dist_2.append(-2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y_temp)+2*6)
        list_similarity_dist_2.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_dist_2.append(rmse_aitchison(Y_temp,dirichRegressor.mu))

Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUC

<timed exec>:13: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


<timed exec>:13: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


...

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


<timed exec>:13: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
ABNORMAL_TERMINATION_IN_LNSRCH


<timed exec>:13: RuntimeWarning: invalid value encountered in divide


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization terminated successfully.
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Optimization t

In [56]:
dirichRegressor.beta

array([[ 0.15765417, -0.85181841, -0.38237642],
       [ 0.15765417, -0.85181841, -0.38237642],
       [-0.33159922, -0.20165317, -0.1646088 ],
       [-0.13844239, -0.21317535, -0.0777844 ]])

In [57]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity", "AIC"])
text_r2 = str(np.round(np.mean(list_r2_ns_2),4))+" ("+str(np.round(np.std(list_r2_ns_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_2),4))+" ("+str(np.round(np.std(list_rmse_ns_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_2),4))+" ("+str(np.round(np.std(list_crossentropy_ns_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_2),4))+" ("+str(np.round(np.std(list_similarity_ns_2),4))+")"
text_aic = str(np.round(np.mean(list_aic_ns_2),4))+" ("+str(np.round(np.std(list_aic_ns_2),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic])

text_r2 = str(np.round(np.mean(list_r2_cont_2),4))+" ("+str(np.round(np.std(list_r2_cont_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_2),4))+" ("+str(np.round(np.std(list_rmse_cont_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_2),4))+" ("+str(np.round(np.std(list_crossentropy_cont_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_2),4))+" ("+str(np.round(np.std(list_similarity_cont_2),4))+")"
text_aic = str(np.round(np.mean(list_aic_cont_2),4))+" ("+str(np.round(np.std(list_aic_cont_2),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic])

text_r2 = str(np.round(np.mean(list_r2_dist_2),4))+" ("+str(np.round(np.std(list_r2_dist_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_2),4))+" ("+str(np.round(np.std(list_rmse_dist_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_2),4))+" ("+str(np.round(np.std(list_crossentropy_dist_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_2),4))+" ("+str(np.round(np.std(list_similarity_dist_2),4))+")"
text_aic = str(np.round(np.mean(list_aic_dist_2),4))+" ("+str(np.round(np.std(list_aic_dist_2),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic])

print(table.draw())

+-------------+------------+------------+------------+------------+------------+
| Model       | $R^2$      | RMSE       | Cross-     | Cos        | AIC        |
|             |            |            | entropy    | similarity |            |
+-------------+------------+------------+------------+------------+------------+
| Not spatial | 0.184      | 0.0994     | 1.1872     | 0.943      | -304.9754  |
|             | (0.0091)   | (0.001)    | (0.0046)   | (0.0014)   | (3.728)    |
+-------------+------------+------------+------------+------------+------------+
| Contiguity  | 0.1841     | 0.0993     | 1.1868     | 0.9431     | -303.4837  |
|             | (0.0082)   | (0.001)    | (0.0049)   | (0.0015)   | (3.8561)   |
+-------------+------------+------------+------------+------------+------------+
| Distance    | 0.1842     | 0.0994     | 1.1872     | 0.943      | -302.9948  |
|             | (0.0088)   | (0.001)    | (0.0046)   | (0.0014)   | (3.7285)   |
+-------------+------------+

In [58]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity", "AIC", "RMSE_A"])
text_r2 = str(np.round(np.mean(list_r2_ns_2),4))+" ("+str(np.round(np.std(list_r2_ns_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_2),4))+" ("+str(np.round(np.std(list_rmse_ns_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_2),4))+" ("+str(np.round(np.std(list_crossentropy_ns_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_2),4))+" ("+str(np.round(np.std(list_similarity_ns_2),4))+")"
text_aic = str(np.round(np.mean(list_aic_ns_2),4))+" ("+str(np.round(np.std(list_aic_ns_2),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_ns_2),4))+" ("+str(np.round(np.std(list_rmse_a_ns_2),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_cont_2),4))+" ("+str(np.round(np.std(list_r2_cont_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_2),4))+" ("+str(np.round(np.std(list_rmse_cont_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_2),4))+" ("+str(np.round(np.std(list_crossentropy_cont_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_2),4))+" ("+str(np.round(np.std(list_similarity_cont_2),4))+")"
text_aic = str(np.round(np.mean(list_aic_cont_2),4))+" ("+str(np.round(np.std(list_aic_cont_2),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_cont_2),4))+" ("+str(np.round(np.std(list_rmse_a_cont_2),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_dist_2),4))+" ("+str(np.round(np.std(list_r2_dist_2),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_2),4))+" ("+str(np.round(np.std(list_rmse_dist_2),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_2),4))+" ("+str(np.round(np.std(list_crossentropy_dist_2),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_2),4))+" ("+str(np.round(np.std(list_similarity_dist_2),4))+")"
text_aic = str(np.round(np.mean(list_aic_dist_2),4))+" ("+str(np.round(np.std(list_aic_dist_2),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_dist_2),4))+" ("+str(np.round(np.std(list_rmse_a_dist_2),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity, text_aic, text_rmse_a])

print(table.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Model     | $R^2$     | RMSE     | Cross-   | Cos simi | AIC      | RMSE_A   |
|           |           |          | entropy  | larity   |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| Not       | 0.184     | 0.0994   | 1.1872   | 0.943    | -304.975 | 1.7482   |
| spatial   | (0.0091)  | (0.001)  | (0.0046) | (0.0014) | 4        | (0.0246) |
|           |           |          |          |          | (3.728)  |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| Contiguit | 0.1841    | 0.0993   | 1.1868   | 0.9431   | -303.483 | 1.7479   |
| y         | (0.0082)  | (0.001)  | (0.0049) | (0.0015) | 7        | (0.0217) |
|           |           |          |          |          | (3.8561) |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| Distance  | 0.1842    | 0.

In [63]:
# For the computation of the AIC, we take the full dataset

dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
dirichRegressor.fit(X_2, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z)
aic_ns = -2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)+2*5
print('AIC not spatial:', aic_ns)

dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor.fit(X_2, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z, W=W_cont)
aic_cont = -2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)+2*6
print('AIC contiguity:', aic_cont, 'with an estimated rho of', dirichRegressor.rho)

dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor.fit(X_2, Y, parametrization='alternative', gamma_0=gamma_0, Z=Z, W=W_dist)
aic_dist = -2*dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)+2*6
print('AIC distance:', aic_dist, 'with an estimated rho of', dirichRegressor.rho)

Optimization terminated successfully.
AIC not spatial: -312.2562710090357
CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
AIC contiguity: -310.2562710080345 with an estimated rho of -0.005062812026629965
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
AIC distance: -310.2564480376951 with an estimated rho of 0.016081539418877147


## Cross-entropy

In [24]:
%%time
list_r2_ns_2_ce, list_rmse_ns_2_ce, list_crossentropy_ns_2_ce, list_similarity_ns_2_ce = [], [], [], []
list_r2_cont_2_ce, list_rmse_cont_2_ce, list_crossentropy_cont_2_ce, list_similarity_cont_2_ce = [], [], [], []
list_r2_dist_2_ce, list_rmse_dist_2_ce, list_crossentropy_dist_2_ce, list_similarity_dist_2_ce = [], [], [], []
list_rmse_a_ns_2_ce, list_rmse_a_cont_2_ce, list_rmse_a_dist_2_ce = [], [], []
list_rho_cont_2_ce, list_rho_dist_2_ce = [], []

for i in range(n):  
    X_temp = np.delete(X_2,i,axis=0)
    Y_temp = np.delete(Y,i,axis=0)
    Z_temp = np.delete(Z,i,axis=0)
    
    Wss_cont = np.delete(W_cont,i,axis=0)
    Wss_cont = np.delete(Wss_cont,i,axis=1)
    Wss_cont = Wss_cont/Wss_cont.sum(axis=1)[:,None]
    
    Wss_dist = np.delete(W_dist,i,axis=0)
    Wss_dist = np.delete(Wss_dist,i,axis=1)
    Wss_dist = Wss_dist/Wss_dist.sum(axis=1)[:,None]
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=False)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, loss='crossentropy')
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_ns_2_ce.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_ns_2_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_ns_2_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_similarity_ns_2_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_ns_2_ce.append(rmse_aitchison(Y_temp,dirichRegressor.mu))

    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_cont, loss='crossentropy')
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_cont_2_ce.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_cont_2_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_cont_2_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_similarity_cont_2_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_cont_2_ce.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
        list_rho_cont_2_ce.append(dirichRegressor.rho)
    
    dirichRegressor = dirichlet_regression.dirichletRegressor(spatial=True)
    dirichRegressor.fit(X_temp, Y_temp, parametrization='alternative', gamma_0=gamma_0, Z=Z_temp, W=Wss_dist, loss='crossentropy')
    if not np.isnan(np.min(dirichRegressor.mu)):
        list_r2_dist_2_ce.append(r2_score(Y_temp,dirichRegressor.mu))
        list_rmse_dist_2_ce.append(mean_squared_error(Y_temp,dirichRegressor.mu,squared=False))
        list_crossentropy_dist_2_ce.append(-1/n*np.sum(Y_temp*np.log(dirichRegressor.mu)))
        list_similarity_dist_2_ce.append(cos_similarity(Y_temp,dirichRegressor.mu))
        list_rmse_a_dist_2_ce.append(rmse_aitchison(Y_temp,dirichRegressor.mu))
        list_rho_dist_2_ce.append(dirichRegressor.rho)

Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization

<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide



Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.


<timed exec>:14: RuntimeWarning: invalid value encountered in divide


Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.


...

Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.


<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide


Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.
NaN result encountered.
Optimization terminated successfully.
Optimization terminated successfully.


<timed exec>:14: RuntimeWarning: invalid value encountered in divide
<timed exec>:14: RuntimeWarning: invalid value encountered in divide


Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Optimization terminated successfully.
Wall time: 3

In [25]:
print('Mean rho contiguity:', np.mean(list_rho_cont_2_ce))
print('Mean rho distance:', np.mean(list_rho_dist_2_ce))

Mean rho contiguity: 0.09472315830229644
Mean rho distance: -0.3006615526683046


In [61]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity"])
text_r2 = str(np.round(np.mean(list_r2_ns_2_ce),4))+" ("+str(np.round(np.std(list_r2_ns_2_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_2_ce),4))+" ("+str(np.round(np.std(list_rmse_ns_2_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_2_ce),4))+" ("+str(np.round(np.std(list_crossentropy_ns_2_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_2_ce),4))+" ("+str(np.round(np.std(list_similarity_ns_2_ce),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_cont_2_ce),4))+" ("+str(np.round(np.std(list_r2_cont_2_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_2_ce),4))+" ("+str(np.round(np.std(list_rmse_cont_2_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_2_ce),4))+" ("+str(np.round(np.std(list_crossentropy_cont_2_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_2_ce),4))+" ("+str(np.round(np.std(list_similarity_cont_2_ce),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity])

text_r2 = str(np.round(np.mean(list_r2_dist_2_ce),4))+" ("+str(np.round(np.std(list_r2_dist_2_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_2_ce),4))+" ("+str(np.round(np.std(list_rmse_dist_2_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_2_ce),4))+" ("+str(np.round(np.std(list_crossentropy_dist_2_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_2_ce),4))+" ("+str(np.round(np.std(list_similarity_dist_2_ce),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity])

print(table.draw())

+-------------+----------------+---------------+---------------+---------------+
| Model       | $R^2$          | RMSE          | Cross-entropy | Cos           |
|             |                |               |               | similarity    |
+-------------+----------------+---------------+---------------+---------------+
| Not spatial | 0.2479         | 0.0961        | 1.1735        | 0.9473        |
|             | (0.0079)       | (0.0009)      | (0.0041)      | (0.0012)      |
+-------------+----------------+---------------+---------------+---------------+
| Contiguity  | 0.2473         | 0.096         | 1.1732        | 0.9474        |
|             | (0.0074)       | (0.0009)      | (0.0045)      | (0.0012)      |
+-------------+----------------+---------------+---------------+---------------+
| Distance    | 0.2472         | 0.0961        | 1.1734        | 0.9473        |
|             | (0.0077)       | (0.0009)      | (0.0041)      | (0.0012)      |
+-------------+-------------

In [62]:
table = Texttable()
table.add_row(["Model", "$R^2$", "RMSE", "Cross-entropy", "Cos similarity", "RMSE_A"])
text_r2 = str(np.round(np.mean(list_r2_ns_2_ce),4))+" ("+str(np.round(np.std(list_r2_ns_2_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_ns_2_ce),4))+" ("+str(np.round(np.std(list_rmse_ns_2_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_ns_2_ce),4))+" ("+str(np.round(np.std(list_crossentropy_ns_2_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_ns_2_ce),4))+" ("+str(np.round(np.std(list_similarity_ns_2_ce),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_ns_2_ce),4))+" ("+str(np.round(np.std(list_rmse_a_ns_2_ce),4))+")"
table.add_row(["Not spatial", text_r2, text_rmse, text_crossentropy, text_similarity, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_cont_2_ce),4))+" ("+str(np.round(np.std(list_r2_cont_2_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_cont_2_ce),4))+" ("+str(np.round(np.std(list_rmse_cont_2_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_cont_2_ce),4))+" ("+str(np.round(np.std(list_crossentropy_cont_2_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_cont_2_ce),4))+" ("+str(np.round(np.std(list_similarity_cont_2_ce),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_cont_2_ce),4))+" ("+str(np.round(np.std(list_rmse_a_cont_2_ce),4))+")"
table.add_row(["Contiguity", text_r2, text_rmse, text_crossentropy, text_similarity, text_rmse_a])

text_r2 = str(np.round(np.mean(list_r2_dist_2_ce),4))+" ("+str(np.round(np.std(list_r2_dist_2_ce),4))+")"
text_rmse = str(np.round(np.mean(list_rmse_dist_2_ce),4))+" ("+str(np.round(np.std(list_rmse_dist_2_ce),4))+")"
text_crossentropy = str(np.round(np.mean(list_crossentropy_dist_2_ce),4))+" ("+str(np.round(np.std(list_crossentropy_dist_2_ce),4))+")"
text_similarity = str(np.round(np.mean(list_similarity_dist_2_ce),4))+" ("+str(np.round(np.std(list_similarity_dist_2_ce),4))+")"
text_rmse_a = str(np.round(np.mean(list_rmse_a_dist_2_ce),4))+" ("+str(np.round(np.std(list_rmse_a_dist_2_ce),4))+")"
table.add_row(["Distance", text_r2, text_rmse, text_crossentropy, text_similarity, text_rmse_a])

print(table.draw())

+-------------+------------+------------+------------+------------+------------+
| Model       | $R^2$      | RMSE       | Cross-     | Cos        | RMSE_A     |
|             |            |            | entropy    | similarity |            |
+-------------+------------+------------+------------+------------+------------+
| Not spatial | 0.2479     | 0.0961     | 1.1735     | 0.9473     | 1.8471     |
|             | (0.0079)   | (0.0009)   | (0.0041)   | (0.0012)   | (0.0302)   |
+-------------+------------+------------+------------+------------+------------+
| Contiguity  | 0.2473     | 0.096      | 1.1732     | 0.9474     | 1.8484     |
|             | (0.0074)   | (0.0009)   | (0.0045)   | (0.0012)   | (0.0286)   |
+-------------+------------+------------+------------+------------+------------+
| Distance    | 0.2472     | 0.0961     | 1.1734     | 0.9473     | 1.8451     |
|             | (0.0077)   | (0.0009)   | (0.0041)   | (0.0012)   | (0.0312)   |
+-------------+------------+